In [4]:
from model.my_models import SiameseUNetWithResnet50Encoder

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, Subset, random_split
from torchvision import transforms
import torchvision.transforms.functional as TF
from torchmetrics import ConfusionMatrix

import math
import os
import numpy as np
from tqdm import tqdm
from PIL import Image
from sklearn.model_selection import train_test_split

from loss_functions import FocalLoss

In [3]:
# in tensorboard see the architecture of model 
# vit对输入图像大小有严格要求，需要是32的倍数，所以这里使用256*256的图像
from torchvision.models import resnet50
#tensorboard
#命令行输入 tensorboard --logdir=./training/runs
from torch.utils.tensorboard import SummaryWriter

writer = SummaryWriter('./runs')
model = SiameseUNetWithResnet50Encoder()

# 定义输入张量，假设输入为两张 256x256 的 RGB 图像
input_1 = torch.rand(1, 3, 256, 256)
input_2 = torch.rand(1, 3, 256, 256)
# 不加torch.no_grad()会报错
with torch.no_grad():
    writer.add_graph(model, (input_1, input_2))
# with torch.no_grad():
#     writer.add_graph(model, input_to_model = torch.rand(1, 3, 256, 256))
writer.close()

In [ ]:

post_dir_img = Path('.\\Dataset\\TierFull\\Post\\Image512\\')
post_dir_mask = Path('.\\Dataset\\TierFull\\Post\\Label512\\')
pre_dir_img = Path('.\\Dataset\\TierFull\\Pre\\Image512\\')
pre_dir_mask = Path('.\\Dataset\\TierFull\\Pre\\Label512\\')
post_dir_val_img = Path('.\\Dataset\\Validation\\Post\\Image512\\')
post_dir_val_mask = Path('.\\Dataset\\Validation\\Post\\Label512\\')
pre_dir_val_img = Path('.\\Dataset\\Validation\\Pre\\Image512\\')
pre_dir_val_mask = Path('.\\Dataset\\Validation\\Pre\\Label512\\')
dir_checkpoint = Path('./checkpoints/Vgg19SiamConc/')

In [ ]:
closs = nn.CrossEntropyLoss()

floss = FocalLoss(mode = 'multiclass',
                alpha = None,
                gamma = 2.0,
                ignore_index = None,
                reduction = "mean",
                normalized = False,
                reduced_threshold = None)